<a href="https://colab.research.google.com/github/brieueu/Projetos_Machine_Learning/blob/main/Cria%C3%A7%C3%A3o_de_um_classificador_de_previs%C3%A3o_de_chuvas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de um classificador de previsão de chuvas
### Projeto final do curso: Aprendizagem automática com Python
Disponível em: https://www.coursera.org/learn/machine-learning-with-python

## Objetivos:
* Explorar e realizar engenharia de atributos em um conjunto de dados do mundo real

* Construir um pipeline de classificação e otimizá-lo usando validação cruzada com busca em grade

* Avaliar seu modelo interpretando várias métricas de desempenho e visualizações

* Implementar um classificador diferente atualizando seu pipeline

* Utilizar um conjunto apropriado de parâmetros para realizar a busca em cada caso


# Sobre o Dataser

A fonte original dos dados é o Bureau of Meteorology do Governo Australiano, e os dados mais recentes podem ser obtidos em http://www.bom.gov.au/climate/dwo/
.

O conjunto de dados utilizados neste projeto foi baixado do Kaggle em https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package/ .
As definições das colunas foram obtidas em http://www.bom.gov.au/climate/dwo/IDCJDW0000.shtml
.

O conjunto de dados contém observações de métricas meteorológicas para cada dia de 2008 a 2017 e inclui os seguintes campos:


| Campo         | Descrição                                             | Unidade         | Tipo   |
| :------------ | :---------------------------------------------------- | :-------------- | :----- |
| Date          | Data da observação no formato AAAA-MM-DD              | Data            | object |
| Location      | Local da observação                                   | Localização     | object |
| MinTemp       | Temperatura mínima                                    | Celsius         | float  |
| MaxTemp       | Temperatura máxima                                    | Celsius         | float  |
| Rainfall      | Quantidade de precipitação                            | Milímetros      | float  |
| Evaporation   | Quantidade de evaporação                              | Milímetros      | float  |
| Sunshine      | Quantidade de luz solar direta                        | Horas           | float  |
| WindGustDir   | Direção da rajada de vento mais forte                 | Pontos cardeais | object |
| WindGustSpeed | Velocidade da rajada de vento mais forte              | Km/h            | object |
| WindDir9am    | Direção do vento (média de 10 min antes das 9h)       | Pontos cardeais | object |
| WindDir3pm    | Direção do vento (média de 10 min antes das 15h)      | Pontos cardeais | object |
| WindSpeed9am  | Velocidade do vento (média de 10 min antes das 9h)    | Km/h            | float  |
| WindSpeed3pm  | Velocidade do vento (média de 10 min antes das 15h)   | Km/h            | float  |
| Humidity9am   | Umidade relativa às 9h                                | Porcentagem     | float  |
| Humidity3pm   | Umidade relativa às 15h                               | Porcentagem     | float  |
| Pressure9am   | Pressão atmosférica ao nível do mar às 9h             | Hectopascal     | float  |
| Pressure3pm   | Pressão atmosférica ao nível do mar às 15h            | Hectopascal     | float  |
| Cloud9am      | Fração do céu coberta por nuvens às 9h                | Oitavos         | float  |
| Cloud3pm      | Fração do céu coberta por nuvens às 15h               | Oitavos         | float  |
| Temp9am       | Temperatura às 9h                                     | Celsius         | float  |
| Temp3pm       | Temperatura às 15h                                    | Celsius         | float  |
| RainToday     | Indica se choveu ao menos 1mm no dia observado        | Sim/Não         | object |
| RainTomorrow  | Indica se choverá ao menos 1mm no dia seguinte        | Sim/Não         | object |


# Baixando Biliotecas

In [1]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install seaborn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

# Carregando dados

* Carregamos os dados.

* Vimos que há muitas faltas em algumas colunas.

* Decidimos remover linhas com valores ausentes.

* Restam 56 mil linhas, o que ainda é suficiente. Mas é preciso cuidado para não usar informações que só estariam disponíveis depois do dia da previsão, senão o modelo ficaria “irrealisticamente bom”.

In [3]:
url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/_0eYOqji3unP1tDNKWZMjg/weatherAUS-2.csv"
df = pd.read_csv(url)
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [8]:
df.count()

,0
Date,145460
Location,145460
MinTemp,143975
MaxTemp,144199
Rainfall,142199
Evaporation,82670
Sunshine,75625
WindGustDir,135134
WindGustSpeed,135197
WindDir9am,134894


In [9]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56420 entries, 6049 to 142302
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           56420 non-null  object 
 1   Location       56420 non-null  object 
 2   MinTemp        56420 non-null  float64
 3   MaxTemp        56420 non-null  float64
 4   Rainfall       56420 non-null  float64
 5   Evaporation    56420 non-null  float64
 6   Sunshine       56420 non-null  float64
 7   WindGustDir    56420 non-null  object 
 8   WindGustSpeed  56420 non-null  float64
 9   WindDir9am     56420 non-null  object 
 10  WindDir3pm     56420 non-null  object 
 11  WindSpeed9am   56420 non-null  float64
 12  WindSpeed3pm   56420 non-null  float64
 13  Humidity9am    56420 non-null  float64
 14  Humidity3pm    56420 non-null  float64
 15  Pressure9am    56420 non-null  float64
 16  Pressure3pm    56420 non-null  float64
 17  Cloud9am       56420 non-null  float64
 18  Cloud3p

In [10]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [11]:
df = df.rename(columns={'RainToday': 'RainYesterday',
                        'RainTomorrow': 'RainToday'
                        })